In [1]:
data Cell = Cell Int Int deriving (Eq, Ord, Show)

coords :: Cell -> (Int,Int)
coords (Cell x y) = (x+1,y+1)

data Group = Vertical Int | Horizontal Int | Square Int Int deriving (Eq, Ord, Show)

groups :: Cell -> [Group]
groups (Cell x y) = [Vertical x, Horizontal y, Square (x `div` 3) (y `div` 3)] 

cells :: Group -> [Cell]
cells (Vertical n)   = [Cell n k | k <- [0..8]]
cells (Horizontal n) = [Cell k n | k <- [0..8]]
cells (Square i j)   = [Cell (3*i+m) (3*j+n) | m <- [0..2], n <- [0..2]]

allCells :: [Cell]
allCells = [Cell i j | j <- [0..8], i <- [0..8]]

allGroups :: [Group]
allGroups = [Vertical x   | x <- [0..8]] ++
            [Horizontal y | y <- [0..8]] ++
            [Square x y   | x <- [0..2]
                          , y <- [0..2]]

In [2]:
import Data.Map (Map)
import qualified Data.Map as M

type Board = Map Cell (Maybe Int)

In [4]:
{-# LANGUAGE ScopedTypeVariables #-}

boardFromString :: String -> Board
boardFromString input = M.fromList (zip allCells initial)
  where
    initial = map (toValue . read) (words input)
    toValue (0 :: Int) = Nothing
    toValue n = Just n


In [5]:
let d = boardFromString "0 0 0 0 6 0 0 0 1 1 0 0 0 0 5 0 6 0 0 6 2 8 3 1 0 0 0 0 8 0 0 0 0 0 9 3 0 0 9 5 0 7 4 0 0 6 5 0 0 0 0 0 7 0 0 0 0 1 7 3 9 2 0 0 1 0 9 0 0 0 0 7 8 0 0 0 5 0 0 0 0"

In [6]:
import Control.Monad
import Data.Maybe

getValue :: Cell -> Board -> Maybe Int
getValue c board = case M.lookup c board of
  Just (Nothing) -> Nothing
  Just (Just n)  -> Just n

showBoard :: Board -> IO ()
showBoard board = do
  let horiz = "+-----------+-----------+-----------+"
  putStrLn horiz
  
  forM_ [0..8] $ \y -> do
    putStr "|"
    forM_ [0..8] $ \x -> do
      putStr $ case getValue (Cell x y) board of
          Nothing -> " . "
          Just n  -> " " ++ show n ++ " "
      putStr (if x `mod` 3 == 2 then "|" else " ")
    putStrLn ""
    when (y `mod` 3 == 2) (putStrLn horiz)
  

In [7]:
showBoard d

+-----------+-----------+-----------+
| .   .   . | .   6   . | .   .   1 |
| 1   .   . | .   .   5 | .   6   . |
| .   6   2 | 8   3   1 | .   .   . |
+-----------+-----------+-----------+
| .   8   . | .   .   . | .   9   3 |
| .   .   9 | 5   .   7 | 4   .   . |
| 6   5   . | .   .   . | .   7   . |
+-----------+-----------+-----------+
| .   .   . | 1   7   3 | 9   2   . |
| .   1   . | 9   .   . | .   .   7 |
| 8   .   . | .   5   . | .   .   . |
+-----------+-----------+-----------+

In [8]:
-- For each cell in the board, gather the set of possible values
import Data.Set (Set)
import qualified Data.Set as S

graph :: (a -> b) -> [a] -> [(a,b)]
graph f xs = zip xs (map f xs)

possible :: Board -> Map Cell (Set Int)
possible board = M.fromList (graph cellPossibilities allCells)
  where
    cellPossibilities c = case getValue c board of
      Just n  -> S.singleton n
      Nothing -> S.fromList [1..9] `S.difference` adjacentValues c board

missing :: Board -> Map Group (Set Int)
missing board = M.fromList (graph groupRequirements allGroups)
  where
    groupRequirements g = S.fromList [1..9] `S.difference` S.fromList (mapMaybe (`getValue` board) (cells g))
    
adjacentValues :: Cell -> Board -> Set Int
adjacentValues c board = S.fromList $ mapMaybe (`getValue` board) (concatMap cells (groups c))

In [9]:
-- For each group, and each missing value in that group:
-- if there is a unique cell in the group that can possibly have that value, then it *must*!

theElement :: Set a -> Maybe a
theElement s = case S.toList s of
  [x] -> Just x
  _   -> Nothing
  
fillRequired :: Board -> Board
fillRequired board = M.mapWithKey fill board
  where
    poss :: Map Cell (Set Int)
    poss = possible board
    
    possibilities :: Cell -> Set Int
    possibilities = fromJust . (`M.lookup` poss)
    
    fill :: Cell -> Maybe Int -> Maybe Int
    fill c v@(Just n) = Just n
    fill c Nothing = listToMaybe $ mapMaybe (fillInGroup c) (groups c)
    
    fillInGroup :: Cell -> Group -> Maybe Int
    fillInGroup c g = theElement $ possibilities c `S.difference`
                                   S.unions (map possibilities (filter (/= c) (cells g)))
    
-- If any cell has 0 possibilities, then return Nothing; else, Just the input board.
isOk :: Board -> Maybe Board
isOk board = if filter S.null (map possibilities allCells) == []
             then Just board
             else Nothing
  where
    poss :: Map Cell (Set Int)
    poss = possible board
    
    possibilities :: Cell -> Set Int
    possibilities = fromJust . (`M.lookup` poss)

In [10]:
toFixpt :: Eq a => (a -> Maybe a) -> a -> Maybe a
toFixpt f x = let mx' = f x in  case mx' of
  Nothing -> Nothing
  Just x' -> if (x' == x) then Just x else toFixpt f x'


In [13]:
import Data.List (sortOn)

solve :: Board -> [Board]
solve board = case toFixpt (isOk . fillRequired) board of
  
  Nothing -> []
  Just board' -> if isSolved board'
                 then [board']
                 else concatMap solve (guesses board')
  where
    
    possibilities :: Board -> Cell -> Set Int
    possibilities b = let poss = possible b in fromJust . (`M.lookup` poss)

    missing b = sortOn (length . snd) [(c, S.toList $ possibilities b c) | (c, Nothing) <- M.toList b]
    
    guesses b = case missing b of
      ((c,gs) : _) -> map (\g -> M.insert c (Just g) b) gs
      [] -> error (show (map (\c -> S.size (possibilities b c)) allCells))
    
    isSolved b = all (\c -> S.size (possibilities b c) == 1) allCells

In [16]:
forM_ (solve d) $ \soln -> do
  showBoard soln
  putStrLn ""

+-----------+-----------+-----------+
| 4   3   5 | 7   6   9 | 2   8   1 |
| 1   7   8 | 4   2   5 | 3   6   9 |
| 9   6   2 | 8   3   1 | 7   4   5 |
+-----------+-----------+-----------+
| 7   8   4 | 2   1   6 | 5   9   3 |
| 3   2   9 | 5   8   7 | 4   1   6 |
| 6   5   1 | 3   9   4 | 8   7   2 |
+-----------+-----------+-----------+
| 5   4   6 | 1   7   3 | 9   2   8 |
| 2   1   3 | 9   4   8 | 6   5   7 |
| 8   9   7 | 6   5   2 | 1   3   4 |
+-----------+-----------+-----------+

+-----------+-----------+-----------+
| 4   3   5 | 7   6   9 | 2   8   1 |
| 1   7   8 | 4   2   5 | 3   6   9 |
| 9   6   2 | 8   3   1 | 7   4   5 |
+-----------+-----------+-----------+
| 7   8   4 | 6   1   2 | 5   9   3 |
| 3   2   9 | 5   8   7 | 4   1   6 |
| 6   5   1 | 3   9   4 | 8   7   2 |
+-----------+-----------+-----------+
| 5   4   6 | 1   7   3 | 9   2   8 |
| 2   1   3 | 9   4   8 | 6   5   7 |
| 8   9   7 | 2   5   6 | 1   3   4 |
+-----------+-----------+-----------+

+---------